### Это файл с конкретными командами для создания и запуска проекта и паука. Работала в Google Colab, поэтому команды выглядят так, как выглядят.

#Продвинутый Python, ДЗ-7

Правила игры:

В домашке 1 задача. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

Оценка за ДЗ складывается следующим образом:

* наличие полного результата - 10 баллов

* наличие только 1 запроса - -4 балла

* отсутствие фильтрации - -2 балла

* запросы только с первой страницы - -2 балла

* не все поля вытащены - -2 балла

В качестве результата добавляйте в репозиторий полностью папку с пауком и всеми необходимыми файлами

**N.B.** Steam спокойно парсится без посторонних инструментов обхода блокировок

## Задание

Парсим [Steam](https://store.steampowered.com/) с помощью Scrapy!

Необходимо выбрать 3 запроса (например, "инди", "стратегии", "minecraft", на каждый запрос должно быть минимум 90 результатов), и вытащить по ним:

* Все игры на первых 2 страницах (найдите, каким образом отображаются страницы)

И для каждой игры вытащить:

* Название

* Категорию (весь путь, за исключением Все игры и самого названия игры)

* Число всех обзоров и общая оценка

* Дата выхода

* Разработчик

* Метки (тэги) игры

* Цена

* Доступные платформы

Сохраните игры в формате json, оставив только игры, выпущенные после 2000 года

Чтобы было проще, в репозитории с ДЗ выложены скрины, где можно посмотреть, где и какие поля находятся

In [1]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.3 MB/s eta 0:00:00


In [2]:
!scrapy startproject spider_steam

New Scrapy project 'spider_steam', using template directory '/usr/local/lib/python3.10/dist-packages/scrapy/templates/project', created in:
    /content/spider_steam

You can start your first spider with:
    cd spider_steam
    scrapy genspider example example.com


In [3]:
!scrapy genspider SteamSpider store.steampowered.com

Created spider 'SteamSpider' using template 'basic' 


In [4]:
!mv SteamSpider.py ./spider_steam/spider_steam/spiders

In [ ]:
#Заменить код в файле SteamSpiders.py на данный

import scrapy
import requests
from bs4 import BeautifulSoup
from lxml import etree
from .. import items
from urllib.parse import urlencode
from urllib.parse import urljoin


class SteamspiderSpider(scrapy.Spider):
    name = "SteamSpider"
    allowed_domains = ["store.steampowered.com"]
    my_categories=["Экшен", "Инди", "Приключения"]
    my_pages=["1", "2"]
    start_urls = []
    for cat in my_categories:
      for p in my_pages:
        curr_url = 'https://store.steampowered.com/search/?' + urlencode({'term': cat, 'ignore_preferences': '1', 'ndl':'1', 'page': p})
        start_urls.append(curr_url)


    def start_requests(self):
        for curr_url in self.start_urls:
            yield scrapy.Request(curr_url, callback=self.parse)

    def parse(self, response):
        game_items = items.SpiderSteamItem()
        root = BeautifulSoup(response.body, 'html.parser')
        dom_root=etree.HTML(str(root))

        game_page_links=dom_root.xpath('//a[@class="search_result_row ds_collapse_flag"]')

        for i in range(len(game_page_links)):
          game_link=game_page_links[i].get("href")

          game_req = requests.get(game_link)
          page = game_req.content.decode("utf-8")
          game = BeautifulSoup(page, 'html.parser')
          dom_game = etree.HTML(str(game))

          release_date=' '.join(dom_game.xpath('//div[@class="app_header_grid_container"]/div[@class="grid_content grid_date"]/text()')[0].split())
          if (release_date[-4:]<'2000'):
            continue

          name=dom_game.xpath('//div[@class="apphub_AppName"]/text()')[0]
          category=dom_game.xpath('//div[@class="blockbg"]/a/text()')[1:]

          reviews_number=""
          overall_grade=""

          marker=dom_game.xpath('//div[@class="user_reviews_summary_row"]')[0].get("data-tooltip-html")
          if (marker=="No user reviews"):
            reviews_number=0
            overall_grade="No user reviews, no grade yet"
          else:
            info=list(map(lambda x: x.strip(), dom_game.xpath('//div[@class="summary column"]/span/text()')))
            if (len(info)>6):
              reviews_number=info[4][1:-1]
              overall_grade=info[3]
            else:
              reviews_number=info[1][1:-1]
              overall_grade=info[0]
            if (reviews_number[0]==" "):
              reviews_number=overall_grade[0]
              overall_grade="Need more user reviews to generate a score"

          developer=dom_game.xpath('//div[@class="app_header_grid_container"]//a/text()')[0]
          tags=list(map(lambda x: x.strip(), dom_game.xpath('//a[@class="app_tag"]/text()')))

          price_res=""
          sample_free=root.find_all(class_="col search_price_discount_combined responsive_secondrow")[i].find(class_="discount_final_price free")
          sample_discount=root.find_all(class_="col search_price_discount_combined responsive_secondrow")[i].find(class_="discount_final_price")
          if (sample_free!=None):
            price_res=sample_free.text
          elif (sample_discount!=None):
            price_res=sample_discount.text
          else:
            price_res="No price yet, game has not been released"

          platforms_raw=dom_game.xpath('//div[@class="sysreq_contents"]/div')
          platforms=[]
          for j in range(len(platforms_raw)):
            platforms.append(platforms_raw[j].get("data-os"))

          game_items['name'] = name
          game_items["category"]=category
          game_items["reviews_number"]=reviews_number
          game_items["overall_grade"]=overall_grade
          game_items["release_date"]=release_date
          game_items["developer"]=developer
          game_items["tags"]=tags
          game_items["price"]=price_res
          game_items["platforms"]=platforms
          yield game_items

In [ ]:
#Заменить код в фале items.py на данный

import scrapy

class SpiderSteamItem(scrapy.Item):
    # define the fields for your item here like:
    name = scrapy.Field()
    category = scrapy.Field()
    reviews_number = scrapy.Field()
    overall_grade = scrapy.Field()
    release_date = scrapy.Field()
    developer = scrapy.Field()
    tags = scrapy.Field()
    price = scrapy.Field()
    platforms = scrapy.Field()

In [ ]:
#Заменить код в файле pipelines.py на данный

import json
from itemadapter import ItemAdapter #класс для обработки items, можно проверить, получили ли мы item

class SpiderSteamPipeline:

    def open_spider(self, spider): # что делать при открытии паука (создаем файлик)
        self.file = open('items.json', 'w')

    def close_spider(self, spider): # что делать при окончании работы паука (закрываем файлик)
        self.file.close()

    def process_item(self, item, spider): #что делать с полученным item
        line = json.dumps(ItemAdapter(item).asdict()) + "\n"
        self.file.write(line)
        return item

In [5]:
!cd spider_steam && scrapy crawl SteamSpider -o items.json

2023-12-18 21:14:49 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: spider_steam)
2023-12-18 21:14:49 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2023-12-18 21:14:49 [scrapy.addons] INFO: Enabled addons:
[]
2023-12-18 21:14:49 [asyncio] DEBUG: Using selector: EpollSelector
2023-12-18 21:14:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-12-18 21:14:49 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-12-18 21:14:49 [scrapy.extensions.telnet] INFO: Telnet Password: 6c3cac0f874d5133
2023-12-18 21:14:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',